In [8]:
from docking.chem import *
from docking.rdock import rdock_virtualscreen
from docking.gnina import gnina_flexdock
import pandas as pd
from tqdm import tqdm

In [17]:
# method = 'flexgnina', 'rdock', 'ledock'
method = 'ledock'
task = 'esmfold_time_split' # 'apo_time_split','esmfold_time_split','holo_time_split'

data_info = pd.read_csv(f'data/{task}.csv')

complex_name = data_info['complex_name'].to_list()
protein_path = data_info['protein_path'].to_list()
crystal_path = data_info['crystal_path'].to_list()
# protein_path = [item.replace('_apo_added.pdb','_apo.pdb') for item in protein_path]
ligand_path = data_info['ligand_path'].to_list()
save_paths = [item.replace('.sdf', f'_{method}.sdf') for item in ligand_path]


dock_info = pd.DataFrame({'complex_name': complex_name, 'protein_path': protein_path, 'crystal_path': crystal_path,'ligand_path': ligand_path, 'save_path': save_paths})

In [20]:
docked_summ_file = osp.join('./data/data_files/',f'{task}_{method}.csv')
if not osp.exists(docked_summ_file):
    dock_info.to_csv(docked_summ_file, index=False)
dock_info = pd.read_csv(docked_summ_file)

In [8]:
import timeout_decorator
@timeout_decorator.timeout(60)
def rdock_iteration(protein_path, ligand_path, save_path):
    protein_mol2 = pdb2mol2(protein_path)
    rdock_virtualscreen(protein_mol2, [ligand_path], ref_ligand=ligand_path, out_file=save_path)

@timeout_decorator.timeout(900)
def gnina_iteration(protein_path, ligand_path, save_path):
    centroid = sdf2centroid(ligand_path)
    gnina_flexdock(protein_path, ligand_path, centroid, out_file=save_path)

from docking.ledock import lepro, ledock_config_generate, generate_ledock_file, docking_with_ledock, dok2sdf
from docking.chem import rmfiles

@timeout_decorator.timeout(60)
def ledock_iteration(protein_path, ligand_path, save_path):
    protein_pro = lepro(protein_path)
    ligand_mol2 = sdf2mol2(ligand_path)
    center = sdf2centroid(ligand_path)
    ligand_list_file, docking_config = ledock_config_generate(protein_pro, ligand_mol2)
    docking_config = generate_ledock_file(protein_pro, center, l_list=[ligand_mol2], l_list_outfile=ligand_list_file, out=docking_config)
    dok_files = docking_with_ledock(docking_config)
    dok_file = dok_files[0]
    result_sdf = dok2sdf(dok_file, ligand_path, save_path)
    rmfiles(*dok_files, docking_config, ligand_list_file, verbose=False) 

In [9]:
for i in tqdm(range(len(dock_info))):
    protein_path = dock_info['protein_path'][i]
    ligand_path = dock_info['ligand_path'][i]
    save_path = dock_info['save_path'][i]
    try: 
        if method == 'rdock':
            rdock_iteration(protein_path, ligand_path, save_path)
        elif method == 'flexgnina':
            gnina_iteration(protein_path, ligand_path, save_path)
        elif method == 'ledock':
            ledock_iteration(protein_path, ligand_path, save_path)
        else:
            raise NotImplementedError
    except timeout_decorator.timeout_decorator.TimeoutError:
        print('timeout')
    except Exception as e:
        print(e)
        
    

  0%|          | 1/353 [00:12<1:11:25, 12.17s/it]

LeDock Docking finished
Have been converted to mol2 file! /home/haotian/Molecule_Generation/Flexible-docking/data/esmfold_time_split/6mhd/6mhd_ligand.sdf


  1%|          | 2/353 [00:21<1:00:38, 10.37s/it]

LeDock Docking finished
Have been converted to mol2 file! /home/haotian/Molecule_Generation/Flexible-docking/data/esmfold_time_split/6i63/6i63_ligand.sdf


  1%|          | 3/353 [00:26<45:41,  7.83s/it]  

...

LeDock Docking finished
Have been converted to mol2 file! /home/haotian/Molecule_Generation/Flexible-docking/data/esmfold_time_split/6cjp/6cjp_ligand.sdf


100%|█████████▉| 352/353 [1:32:03<00:11, 11.61s/it]

LeDock Docking finished
Have been converted to mol2 file! /home/haotian/Molecule_Generation/Flexible-docking/data/esmfold_time_split/6nri/6nri_ligand.sdf


100%|██████████| 353/353 [1:32:15<00:00, 15.68s/it]

LeDock Docking finished
